In [ ]:
from __future__ import print_function
import os
import datetime

from google.auth.transport.requests import Request
from google.oauth2.credentials import Credentials
from google_auth_oauthlib.flow import InstalledAppFlow
from googleapiclient.discovery import build

In [16]:
SCOPES = ["https://www.googleapis.com/auth/calendar.readonly"]
def get_calendar_service():
    creds = None
    # token.json stores the user's access and refresh tokens
    if os.path.exists("token.json"):
        creds = Credentials.from_authorized_user_file("token.json", SCOPES)
    # If there are no valid creds, let user log in
    if not creds or not creds.valid:
        if creds and creds.expired and creds.refresh_token:
            creds.refresh(Request())
        else:
            flow = InstalledAppFlow.from_client_secrets_file("/home/timmy/ai-teacher-assistant/credentials.json", SCOPES)
            creds = flow.run_local_server(port = 8080)
        # Save creds for next run
        with open("token.json", "w") as token:
            token.write(creds.to_json())
    return build("calendar", "v3", credentials=creds)

In [17]:
# Test: List next 10 events
service = get_calendar_service()
now = datetime.datetime.utcnow().isoformat() + "Z"
print("Getting upcoming 10 events")
events_result = service.events().list(calendarId="primary", timeMin=now,
                                      maxResults=10, singleEvents=True,
                                      orderBy="startTime").execute()
events = events_result.get("items", [])

Please visit this URL to authorize this application: https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=840970484260-019tusa2dnknt9s8445n1m0185sl1qn8.apps.googleusercontent.com&redirect_uri=http%3A%2F%2Flocalhost%3A8080%2F&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fcalendar.readonly&state=oYbwqvoQf76aPeAxWTJHpIykPYc2KN&access_type=offline


gio: https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=840970484260-019tusa2dnknt9s8445n1m0185sl1qn8.apps.googleusercontent.com&redirect_uri=http%3A%2F%2Flocalhost%3A8080%2F&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fcalendar.readonly&state=oYbwqvoQf76aPeAxWTJHpIykPYc2KN&access_type=offline: Operation not supported
/tmp/ipykernel_50360/3816084315.py:3: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  now = datetime.datetime.utcnow().isoformat() + "Z"


Getting upcoming 10 events


In [ ]:
if not events:
    print("No upcoming events found.")
for event in events:
    start = event["start"].get("dateTime", event["start"].get("date"))
    print(start, event["summary"])